# Bootstrap generate album pages with tags

Data Sources
- https://www.flaticon.com

## Setup

### Import modules

In [1]:
import os
import pandas
import bs4
import math
import datetime
import random

### Set html colour codes 

In [2]:
jumbotron_background = "#41a8ba"
jumbotron_foreground = "#FFFFFF"
outline_button_foreground = "danger"
card_header_foreground = "#41a8ba"
navbar_background = "#414242"
footer_background = "#414242"
badge_pill_background = "#414242"
badge_pill_foreground  = "#FFFFFF"

### Set SVG flag, items per page

In [3]:
items_per_page = 9

svg = True

if svg == True:
    visual_tag_start = '<object class = "p-3" type="image/svg+xml" data="'
    visual_tag_end = '">Your browser does not support SVG</object>'
else:
    visual_tag_start = '<img class="card-img-top p-3" src="'
    visual_tag_end = '" alt="Card image cap">'

### Set Data Path

In [4]:
data_path = "/users/danielcorcoran/Desktop/github_repos/bootstrap_generate/bus_svg/"

In [5]:
data_path_items = data_path.split("/")
data_path_short = data_path_items[len(data_path_items) - 2] + "/"
data_path_short

'bus_svg/'

## Read in files

### Read filenames from data_path variable

In [6]:
all_filenames = os.listdir(data_path)

### Store filenames in dataframe

In [7]:
data = pandas.DataFrame()
data["filename"] = pandas.Series(all_filenames)

### Create tags for testing purposes

In [8]:
test_tags = ["blue", "red", "yellow", "purple", "grey", "white", "black", "teal", "orange",
            "silver", "pink", "brown"]

In [9]:
for index in range(data.shape[0]):
    
    temp_tags = ""
    number_of_tags = random.randint(1,7)    
    
    for sub_index in range(number_of_tags):
        tag_index = random.randint(0,len(test_tags)-1)
        test_tag = test_tags[tag_index]
        
        if temp_tags == "":
            temp_tags = temp_tags + test_tag
        else:
            temp_tags = temp_tags + "," +test_tag
    
    data.loc[index, "tag"] = temp_tags

## Extract tags from dataframe

In [10]:
tags = data["tag"].tolist()

In [11]:
tagstring =""

for tag in tags:
    if tag is not None:
        tagstring = tagstring +","+ tag

In [12]:
tagstring = tagstring.lower().strip()

In [13]:
many_tags = tagstring.split(",")

### Clean tags, lowercase and strip

In [14]:
for index in range(len(many_tags)):
    many_tags[index] = many_tags[index].lower().strip()

In [15]:
for item in many_tags:
    if len(item) == 0:
        many_tags.remove(item)

In [16]:
no_dupes = []

for item in many_tags:
    if item not in no_dupes:
        no_dupes.append(item)

### Create dictionary to store indices for each tag

In [17]:
dic = {}
for tag in no_dupes:
    dic[tag] = []

for tag in no_dupes:
    for index in range(data.shape[0]):
        current_tag = data.loc[index, "tag"]
        if tag in current_tag:
            dic[tag].append(index)

dic["index"] = list(data.index)

dic

{'purple': [0,
  2,
  6,
  7,
  8,
  10,
  11,
  14,
  16,
  17,
  20,
  24,
  25,
  30,
  34,
  42,
  43,
  46,
  47,
  62,
  71,
  73,
  74,
  77,
  82,
  83,
  104,
  106,
  114,
  122,
  131,
  132,
  136,
  137,
  138,
  145,
  152,
  155],
 'orange': [0,
  3,
  7,
  11,
  15,
  16,
  20,
  24,
  26,
  28,
  29,
  35,
  36,
  38,
  40,
  43,
  46,
  54,
  62,
  66,
  67,
  68,
  69,
  70,
  71,
  73,
  76,
  78,
  82,
  83,
  85,
  86,
  94,
  95,
  97,
  102,
  103,
  106,
  108,
  109,
  124,
  133,
  134,
  137,
  144,
  150,
  153,
  157,
  159],
 'pink': [0,
  4,
  20,
  22,
  28,
  29,
  33,
  35,
  37,
  46,
  49,
  50,
  60,
  62,
  67,
  70,
  74,
  80,
  84,
  87,
  93,
  94,
  95,
  99,
  109,
  111,
  113,
  119,
  120,
  121,
  123,
  124,
  129,
  130,
  132,
  142,
  143,
  145,
  146,
  153,
  155,
  156,
  157,
  158,
  159,
  166,
  167],
 'yellow': [1,
  2,
  4,
  5,
  6,
  8,
  9,
  14,
  16,
  23,
  24,
  27,
  43,
  45,
  47,
  49,
  50,
  51,
  53,
  55,
  5

## Html Skeletons

### Jumbotron

In [18]:
jumbotron_home = '''
<div class="jumbotron jumbotron-fluid text-center" style = "color: '''+jumbotron_foreground+'''; background-color:'''+jumbotron_background+''';">
      <div class="container">
        <h1 class="display-2">
          Daniels Generated Album
        </h1>
        <p class="lead">
          Something short and leading about the collection below—its contents, the creator, etc. Make it short and sweet, but not too short so folks don't simply skip over it entirely.
        </p>
        <p>
          <a class="btn btn-outline-light my-2" href="#">
          Primary
          </a>
          <a class="btn btn-outline-light my-2" href="#">
          Secondary
          </a>
        </p>
      </div>
    </div>
'''

jumbotron_empty = ''

### Pagination - Variable

In [19]:
pagination = '<div class = "text-center pb-4 text-muted bg-light">\
  <div class = "btn-group">\
    <a href = "{}{}.html" class = "btn btn-outline-primary {}">First</a>\
    <a href = "{}{}.html" class = "btn btn-outline-primary {}">Previous</a>\
    <a class = "btn btn-outline-primary active">Page {}</a>\
    <a href = "{}{}.html" class = "btn btn-outline-primary {}">Next</a>\
    <a href = "{}{}.html"class = "btn btn-outline-primary {}">Last</a>\
  </div>\
</div>'

pagination2 = '''
<div class = "text-center pb-4 text-muted bg-light">
  <div class = "btn-group">
    <a class = "btn btn-outline-primary active">Page {}</a>
  </div>
</div>
'''

### Skeleton

In [20]:
skeleton = '''
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8"/>
    <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
    <meta content="" name="description"/>
    <meta content="" name="author"/>
    <link href="../../../../favicon.ico" rel="icon"/>
    <title>
      Daniels Generated Gallery
    </title>
    <!-- Bootstrap CSS -->
    <link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/css/bootstrap.min.css" integrity="sha384-WskhaSGFgHYWDcbwN70/dfYBj47jz9qbsMId/iRN3ewGhXQFZCSftd1LZCfmhktB" rel="stylesheet"/>
    <!-- Custom styles for this template -->
    <link href="album.css" rel="stylesheet"/>
  </head>
  <body class = "bg-light">
    <header style = "background-color: '''+navbar_background+''';">
      <div class = "container">
      <nav class="navbar navbar-expand-lg navbar-dark py-3" style = "background-color: '''+navbar_background+''';">
        <a class="navbar-brand" href="#">
        Gallery
        </a>
        <button aria-controls="navbarColor03" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler collapsed" data-target="#navbarColor03" data-toggle="collapse" type="button">
        <span class="navbar-toggler-icon">
        </span>
        </button>
        <div class="navbar-collapse collapse" id="navbarColor03" style="">
          <ul class="navbar-nav mr-auto">
            <li class="nav-item active">
              <a class="nav-link" href="index_0.html">
              Home
              <span class="sr-only">
              (current)
              </span>
              </a>
            </li>
            <li class="nav-item">
              <a class="nav-link" href="#">
              About
              </a>
            </li>
            <li class="nav-item">
              <a class="nav-link" href="#">
              Contact Us
              </a>
            </li>
          </ul>
        </div>
      </nav>
      </div>
    </header>
    
    
    {}
    
    
    
    <div class="album bg-light">
      <div class="container">
        <div class="row">
          
          {}
        
        
        </div>
      </div>
    </div>
    
    {}
    
    <footer class="text-light py-3" style = "background-color: '''+footer_background+''';">
      <div class="container">
        <span>&copy; Copyright DanielsGenerator</span>
        <span class="float-right">
        <a href="#">
        Back to top
        </a>
        </span>
      </div>
    </footer>
    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script crossorigin="anonymous" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
    <script crossorigin="anonymous" integrity="sha384-ZMP7rVo3mIykV+2+9J3UJ46jBk0WLaUAdn689aCwoqbBJiSnjAK/l8WvCWPIPm49" src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.3/umd/popper.min.js"></script>
    <script crossorigin="anonymous" integrity="sha384-smHYKdLADwkXOn1EmN1qk/HfnUcbVRZyYmZ4qpPea6sjB/pTJ0euyQp0Mk8ck+5T" src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/js/bootstrap.min.js"></script>
  </body>
</html>
'''

## Process

In [21]:
for key in dic.keys():
    
    #SET TAG
    tag = key
    page_prefix = tag
    print("processing dic key",tag)
    subset_indices = dic[tag]

    #WORK OUT TOTAL ITEMS
    total_items = len(dic[tag])
    total_items
    
    #WORK OUT HOW MUCH EXCESS ITEMS
    excess_items = total_items % items_per_page
    
    # CALCULATE HOW MANY PAGES WILL BE REQUIRED
    if excess_items > 0:
        total_pages = int(math.floor(total_items/items_per_page) + 1)
    else:
        total_pages = int(total_items/items_per_page)
        
    index_starter = 0
    nested_page_indices = []
    # CALCULATE GROUPINGS OF FILES PER PAGE AND STORE IN A LIST OF LISTS

    if total_pages > 0:
        for index in range(total_pages):
            small_list = subset_indices[index_starter: index_starter + items_per_page]
            nested_page_indices.append(small_list)
            index_starter = index_starter + items_per_page
    
    #CALCULATE INDEX FOR FIRST, LAST PAGES, SET DISABLED VARIABLES
    first_index = 0
    last_index = len(nested_page_indices) - 1
    nothing = ''
    disabled = 'disabled'
    
    for index in range(len(nested_page_indices)):
    
        # WORK OUT INDEX FOR PAGES LATERALLY
        previous_index = index -1
        next_index = index +1

        body_final = ''
        page_items = nested_page_indices[index]
        page_number = index + 1

        #PAGINATION
        if total_pages == 1:
            pagination_final = pagination2.format(page_number)
        else:
            if index ==0:
                pagination_final = pagination.format(page_prefix, first_index,disabled,page_prefix,  previous_index, disabled, page_number,
                                                    page_prefix, next_index, nothing, page_prefix, last_index, nothing)
            elif index == last_index:
                pagination_final = pagination.format(page_prefix, first_index, nothing, page_prefix, previous_index, nothing, page_number,
                                                    page_prefix, next_index, disabled, page_prefix, last_index, disabled)
            else:
                pagination_final = pagination.format(page_prefix,first_index, nothing,page_prefix, previous_index, nothing, page_number,
                                                    page_prefix, next_index, nothing, page_prefix, last_index, nothing)

        if index == 0:
            jumbotron_final = jumbotron_home
        else:
            jumbotron_final = jumbotron_empty

        # FILE LEVEL LOOP

        page_level_indices = nested_page_indices[index]

        item_number = 1

        for sub_index in page_level_indices:

            filename = data.loc[sub_index, "filename"]
            filetags = data.loc[sub_index, "tag"]

            tags_split = filetags.split(",")
            tags_html_block = ""

            for tag_name in tags_split:
                tags_html_block = tags_html_block + \
                '<a href = "'+tag_name+'0.html" class = "badge badge-pill" style = "color:'+badge_pill_foreground+' ;background-color:'+badge_pill_background+';">'+tag_name+'</a>'

            # IF ITEM IS ON FIRST ROW SET TOP MARGIN TO 4 ELSE NO MARGIN
            if item_number <=3:
                optional_top_margin = 'mt-4'
            else:
                optional_top_margin = ''

            item_number = item_number + 1

            filename_short = filename.replace(".png", "")

            body_final = body_final +'<div class="col-lg-4 col-md-6">\
                      <div class="card mb-4 ' + optional_top_margin+'">\
                      ' + visual_tag_start + data_path_short + filename + visual_tag_end + '\
                        <div class="card-body">\
                          <h4 class = "text-primary">'+filename_short+'</h4>\
                          <p class="card-text">This is a wider card with supporting text below as a natural lead-in to additional content. This content is a little bit longer.</p>\
                            <div class="btn-group p-0 m-0">\
                              <a class="btn btn-sm btn-outline-primary" href="'+data_path_short + filename+'" role="button">View</a>\
                              <a class="btn btn-sm btn-outline-primary" href = "'+data_path_short + filename+'" download = "'+filename+'" role="button">Download</a>\
                            </div>\
                        </div>\
                        <div class = "p-3 bg-light">\
                          '+tags_html_block+'\
                        </div>\
                      </div>\
                    </div>'

        # BUILD FULL HTML CODE ON PAGE LEVEL
        full = skeleton.format(jumbotron_final, 
                               body_final, 
                               pagination_final)

        # PRETTIFY/FORMAT THE FULL HTML CODE
        full_soup = bs4.BeautifulSoup(full, "lxml")
        full_pretty = full_soup.prettify()

        #EXPORT HTML
        with open("/users/danielcorcoran/desktop/github_repos/bootstrap_generate/" +tag+str(index)+".html", "w") as fileobject:
            fileobject.write(full_pretty)

        fileobject.close()

processing dic key purple
processing dic key orange
processing dic key pink
processing dic key yellow
processing dic key blue
processing dic key red
processing dic key white
processing dic key teal
processing dic key black
processing dic key silver
processing dic key brown
processing dic key grey
processing dic key index
